In [58]:
# Import libs
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [59]:
# caminho para a pasta raiz que contém as pastas de dados (train, test e val)
data_dir = 'database/'

In [60]:
# define as transformações de dados que você deseja aplicar às imagens
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # redimensiona a imagem para 224x224 pixels
    transforms.ToTensor(),          # converte a imagem em um tensor PyTorch
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # normaliza os valores dos pixels da imagem
])


In [61]:
# carrega os dados de treinamento, teste e validação usando ImageFolder
train_data = ImageFolder(root=data_dir + '/train', transform=transform)
test_data = ImageFolder(root=data_dir + '/test', transform=transform)
val_data = ImageFolder(root=data_dir + '/val', transform=transform)

In [62]:
# imprime o número de imagens em cada conjunto de dados
print(f"Number of images in training set: {len(train_data)}")
print(f"Number of images in test set: {len(test_data)}")
print(f"Number of images in validation set: {len(val_data)}")

# imprime as classes disponíveis nos dados
print(f"Classes: {train_data.classes}")

Number of images in training set: 5216
Number of images in test set: 624
Number of images in validation set: 16
Classes: ['NORMAL', 'PNEUMONIA']


In [63]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.fc1 = nn.Linear(128 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [64]:
# Define o modelo e o mova para o dispositivo (GPU ou CPU)
model = Model()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
# Define a função de perda (loss function)
criterion = nn.BCELoss()

# Define o otimizador
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define o número de épocas de treinamento
num_epochs = 10

# Cria os dataloaders para os dados de treinamento e validação
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)



In [66]:
# Treina o modelo
for epoch in range(num_epochs):
    # Treinamento
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    train_loss /= len(train_loader.dataset)
    
    # Validação
    model.eval()
    val_loss = 0.0
    val_correct = 0
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        val_loss += loss.item() * inputs.size(0)
        predicted = torch.round(outputs)
        val_correct += (predicted == labels.float().unsqueeze(1)).sum().item()
    val_loss /= len(val_loader.dataset)
    val_accuracy = val_correct / len(val_loader.dataset)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x86528 and 18432x128)